# python3使用教程

python是非常主流的一门语言，常被用在网站，爬虫，科学计算等领域

## python文件

**标准注释：**
```python
#!/usr/bin/env python3 
# -*- coding: utf-8 -*-
```
第一行作为通用环境的标准写法，保证在不同环境下正常执行，比如window和linux。第二行指定编码格式

## python模块

python模块的组织以文件或文件夹作为模块。**模块**作为一组函数与变量等符号的集合对外提供，来增加代码可复用性。模块的另一个作用是通过命名空间的方式，来组织不同功能的符号，以解决命名冲突。通常使用文件夹的方式称为包。

包是以目录树的方式来组织模块的一种方式。内部包含子模块和子包

```bash
example           # 包
  |-- __init__.py  # 标志这是一个包
  |-- a.py
  |-- b.py
  |-- sub-lib     # 子包
        |-- __init__.py
        |-- c.py
```

__init__.py 一般内容。 文件也可以加这段代码，以作为可执行脚本执行。
```python
if __name__ == '__main__':
    print('作为可执行程序运行')
else:
    print('作为包初始化执行')
```


#### 模块的使用

```python
from bs4 import BeautifulSoup # 引入bs4模块的BeautifulSoup符号
import requests # 引入requests模块
# from requests import * # 引入所有符号
# import filename # 通过文件名引入
requests.get('http://baidu.com') # 使用模块名来访问内部的共有变量
```

#### 模块的创建

使用文件的方式来作为一个模块，或者报的方式来作为一个模块使用

#### 模块的选择

1. 当前目录
2. PYTHONPATH环境变量
3. python默认路径，比如/usr/local/lib/python

#### 符号的查看

> dir(module)  查看当前module模块内的符号
>
> globals()  查看当前位置能访问的全局变量
>
> locals()  查看当前位置能访问的局部变量

In [16]:
#!/usr/bin/env python3 
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup # 引入bs4模块的BeautifulSoup符号
import requests # 引入requests模块
resp = requests.get('https://baidu.com') # 使用模块名来访问内部的共有变量
# print(resp.text)

## python的使用

#### 标识符

1. 标识符有字母，数字，下划线组成。不易数字开头
2. 以__开头标识私有变量，如 __foo
3. 以__开头和结尾表示特殊变量，如 __init__

#### 缩进与换行

python使用缩进与换行来表示语句结束，或块范围。同一行有多个语句时才使用; 

## 使用python爬取小说

使用requests爬取顶点小说https://www.ddxstxt8.com/

1. 分析网站，发现有全部小说tab，收集了该网站的所有小说 /xiaoshuodaquan
2. 小说列表是服务端直接渲染的html页面。
3. 小说详情页有小说章节列表

ps: 注意使用代理池以防止ip被封

In [111]:
#!/usr/bin/env python3 
# -*- coding: utf-8 -*-

import requests
from bs4 import BeautifulSoup
import re

ua = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36'
}

def get_el_value(elements,  type='string', index = 0):
    if index < len(elements):
        if type == 'string':
            return elements[index].string
        elif type == 'text':
            return elements[index].get_text()
        else:
            return elements[index][type]
    else:
        return ''
    
def url_concat(path, href):
    prefix = re.match(r'^(https?://.*)/', path).group(1)
    root = re.match(r'^(https?://.*?)/', path).group(1)
    url = prefix
    
#     print('href', path, href, prefix, root)
    if href.startswith('/'):
        url = root + href
    elif (href.startswith('./')):
        url = prefix + href[1:]
    else:
        url = prefix + '/' + href
    return url


def get_novel_list():
    resp = requests.get('https://www.ddxstxt8.com/xiaoshuodaquan/', headers=ua)
    soup = BeautifulSoup(resp.text, 'lxml')
    data = soup.select('#main > div.novellist > ul > li > a')

    novels = []
    for item in data:
        name = item.string
        link = item['href']
        novels.append({
            'name': name,
            'link': link
        })
    return novels

# url = 'https://www.ddxstxt8.com/4_4891/'
def get_novel(url):
    chapters = []
    resp = requests.get(url, headers=ua)
    resp.encoding = "gbk"
    soup = BeautifulSoup(resp.text, 'lxml')
    
    info = get_el_value(soup.select('#info > p:nth-child(2)'))
    author = re.sub(r'^作.*者：', '', info)
    novel_name = get_el_value(soup.select('#info > h1'))
    desc = get_el_value(soup.select('#intro > p'))
    img_path = get_el_value(soup.select('#fmimg > img'),'src')
    img = url_concat(url, img_path)

    data = soup.select('#list > dl > dd > a')[6:]
    for item in data:
        name = item.string
        href = item['href']
        chapters.append({
            'chapter_name': name,
            'link': url + href
        })
#     print(info, desc, name, img)
    return {
        'novel_name': novel_name,
        'menu_link': url,
        'image': img,
        'author': author,
        'desc': desc,
        'chapters': chapters
    }


# url = 'https://www.ddxstxt8.com/1_1651/11057637.html'
def get_article(url):
    resp = requests.get(url, headers=ua)
    resp.encoding = "gbk"
    soup = BeautifulSoup(resp.text, 'html5lib')

    name = get_el_value(soup.select('#wrapper > div.content_read > div > div.bookname > h1'))

    text = get_el_value(soup.select('#content'), 'text')
#     text = soup.select('#content')[0].get_text()
    text = text.split('chaptererror()')[0]
#     print(name, text)
    return {
        'article_name': name,
        'article_text': text
    }

In [87]:
novels = get_novel_list()
# print('novels:', novels)

In [88]:
novel = get_novel(novels[0]['link'])
# print('novel:', novel)

In [113]:
article_content = get_article('https://www.ddxstxt8.com/1_1651/11057637.html')
# print(article_content['article_text'])

## 添加代理池防止IP被封

ip代理池可以将我们的数据包分流发出，服务端通过http层的Reffer拿到的来源是经过一跳的。代理池甚至可以对这层进行处理，使之只有一个来源。以防止ip被封。

使用商用ip代理池，更稳定，更方便。免费ip代理池基本没有可用ip



In [126]:
import random
import time
def get_proxy():
    def get_ips():
        resp = requests.get('http://api.wandoudl.com/api/ip?app_key=0093c1a82bf9136e2222b88fdf78c6c2&pack=0&num=20&xy=1&type=2&lb=\r\n&mr=2&area_id=undefined')
        body = resp.json()
        if body['code'] != 200:
            return []
        return body['data']
    proxys = get_ips()
    if len(proxys) == 0:
        print('重新请求...')
        proxys = get_ips()
    if len(proxys) > 0
        proxy = proxys[random.randint(0, len(proxys)-1)]
        if time.mktime(time.strptime(proxy['expire'], '') <= time.time()):
#             proxys.remove('') # 移除失效ip
            
    print('resp', resp)
get_proxy()